In [1]:
COLLECTION_NAME = 'movies_vecdb'
DIMENSION = 384
URI = 'http://localhost:19530'

In [2]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores.milvus import Milvus
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [3]:
import os

docs = []

for file in os.listdir('./movie_text_data/'):
    loader = TextLoader(f'./movie_text_data/{file}')
    docs.extend(loader.load())

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=0)  # DA OTTIMIZZARE
all_splits = text_splitter.split_documents(docs)

In [4]:
import torch

device = None
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

model_name = "sentence-transformers/all-MiniLM-L12-v2"
model_kwargs = {'device': device}

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

In [5]:
vector_db = Milvus(
    embedding_function=embeddings,
    connection_args={'host':'127.0.0.1', 'port':'19530'},
    drop_old=False
)

In [6]:
vector_db = Milvus.from_documents(
    all_splits,
    embeddings,
    connection_args={'host':'127.0.0.1', 'port':'19530'},
)

In [7]:
query = "Which are the Crime movies directed by Martin Scorsese?"
result = vector_db.similarity_search(query, k=3)

In [8]:
for n, doc in enumerate(result):
    print(f"DOCUMENT {n+1}:")
    print(doc.page_content)
    print()

DOCUMENT 1:
IMDB ID -> 0101540
TITLE -> Cape Fear
DIRECTED BY -> Martin Scorsese
GENRES -> Crime, Thriller
PLOT -> A convicted rapist, released from prison after serving a fourteen-year sentence, stalks the family of the lawyer who originally defended him.
YEAR -> 1991
STARS -> Robert De Niro, Nick Nolte, Jessica Lange

DOCUMENT 2:
IMDB ID -> 0031014
TITLE -> 6,000 Enemies
DIRECTED BY -> George B. Seitz
GENRES -> Crime, Drama, Romance
PLOT -> A tough prosecutor who has sent dozens of criminals to prison finds himself framed on a bribery charge and winds up in prison himself.
YEAR -> 1939
STARS -> Walter Pidgeon, Rita Johnson, Paul Kelly

DOCUMENT 3:
IMDB ID -> 0031014
TITLE -> 6,000 Enemies
DIRECTED BY -> George B. Seitz
GENRES -> Crime, Drama, Romance
PLOT -> A tough prosecutor who has sent dozens of criminals to prison finds himself framed on a bribery charge and winds up in prison himself.
YEAR -> 1939
STARS -> Walter Pidgeon, Rita Johnson, Paul Kelly



In [9]:
print(f"Default collection name - {vector_db.collection_name}")
print(f"Default search params - {vector_db.search_params}")
print(f"Default index params - {vector_db.index_params}")

Default collection name - LangChainCollection
Default search params - {'metric_type': 'L2', 'params': {'ef': 10}}
Default index params - None


In [10]:
vector_db.col

<Collection>:
-------------
<name>: LangChainCollection
<description>: 
<schema>: {'auto_id': True, 'description': '', 'fields': [{'name': 'source', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 65535}}, {'name': 'text', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 65535}}, {'name': 'pk', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': True}, {'name': 'vector', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 384}}], 'enable_dynamic_field': False}

In [11]:
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_core.prompts import PromptTemplate

In [91]:
llm = LlamaCpp(
        model_path="./models/Meta-Llama-3-8B-Instruct-Q6_K.gguf",
        n_gpu_layers=-1,
        n_batch=1024,
        temperature=0.2,
        n_threads=8,
        # max_tokens=16384,
        # f16_kv=True,
        # callback_manager=CallbackManager(StreamingStdOutCallbackHandler()),
        verbose=True
)

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from ./models/Meta-Llama-3-8B-Instruct-Q6_K.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = Meta-Llama-3-8B-Instruct
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.attention.head_count u32              = 32
llama_model_loader: - kv   7:              llama.att

In [103]:
from langchain_core.prompts import PromptTemplate

template = """<|begin_of_text|>
<|start_header_id>system<|end_header_id>
You are an helpful AI assistant, which is expert about the field of Movies and Cinema.
You have to answer to the question in a precise way: you MUST use the search results to build the answer.
If the asked informations don't exist within the search results, DON'T INVENT THEM AND JUST ANSWER 'I don't know how to answer to this question'. 

Generate some movies and, for each movie, report these informations following this structure:
* Title: report here the title of the movie
* Year: report here the year of the movie
* Genre: report here the list of genres for the movie
* Director: report here the director 
* Brief description of the plot: report here a brief description of the plot
* Main actors: report here 2 or 3 members of the cast
<|eot_id|>

<|start_header_id>user<|end_header_id>
Question: {question}
<|eot_id|>

<start_header_id>assistant<|end_header_id>
Answer: """

prompt = PromptTemplate.from_template(template)

In [104]:
from langchain_core.runnables import RunnablePassthrough

retriever = vector_db.as_retriever()

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()} | prompt | llm
)

In [117]:
request = """Report me some movies which correspond to at least one of these features: the movie directed by Pedro Almodovar, the plot is related to the Vietnam War, the movie is less recent than 1988, Brad Pitt is part of the cast.
Give more importance to a movie if it corresponds to more than one of the listed features.
After reporting the movies, explain why you chose them."""

resp = rag_chain.invoke(request)
print(str(resp))

Llama.generate: prefix-match hit

llama_print_timings:        load time =     393.71 ms
llama_print_timings:      sample time =     296.69 ms /   205 runs   (    1.45 ms per token,   690.97 tokens per second)
llama_print_timings: prompt eval time =     293.55 ms /    58 tokens (    5.06 ms per token,   197.58 tokens per second)
llama_print_timings:        eval time =    5087.63 ms /   204 runs   (   24.94 ms per token,    40.10 tokens per second)
llama_print_timings:       total time =    7209.53 ms /   262 tokens


 Here are some movies that correspond to at least one of the listed features:

1. **Tortilla Soup** (2001) - Directed by Pedro Almodovar's protégé, Robert Rodriguez. The movie is a remake of Almodovar's **Like Water for Chocolate**.
2. **We Were Soldiers** (2002) - The plot is related to the Vietnam War.
3. **The Thin Red Line** (1998) - Although not directly about the Vietnam War, it is set during World War II and explores themes of war and humanity.
4. **Se7en** (1995) - Directed by David Fincher, who has worked with Brad Pitt on several projects.

I chose these movies because they all have connections to at least one of the listed features. For example, **Tortilla Soup** is a remake of Pedro Almodovar's movie, and **We Were Soldiers** is a war drama that explores themes related to the Vietnam War.
